# Welcome to the 2024 workshop!

Read carefully the instructions throughout this notebook. They will guide you in the realm of industry-level
anomaly detection problem. It will be instructive and (hopefully!) fun.

The initial cells are fully working and ready to be run. Try to understand what each cell does, comments can
help you during the process. Later on, there will be some parts that will require your intervention to work as
expected.

In [ ]:
# This cell can be ignored. It will be required in case you need to setup an environment from scratch
# (e.g. if you are using Colab).

# !pip install anomalib imgaug kornia lightning matplotlib 'numpy<=2.0.0' onnxruntime opencv-python pandas pillow scikit-learn torch torchvision timm FrEIA open-clip-torch

These are the "imports" for the notebook. You could see some warnings, but they are safe to ignore.
Simply run this cell and move on.

In [ ]:
import random
from collections import defaultdict
from typing import Any, Dict, List, Tuple

import cv2
import lightning as L
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import torch
from anomalib import TaskType
from anomalib.data import BTech, MVTec
from anomalib.data.utils import read_image
from anomalib.engine import Engine
from anomalib.metrics import F1AdaptiveThreshold, ManualThreshold
from anomalib.metrics.precision_recall_curve import BinaryPrecisionRecallCurve
from anomalib.models import Padim, Fastflow, Patchcore, Uflow
from anomalib.utils.post_processing import superimpose_anomaly_map, anomaly_map_to_color_map
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from matplotlib import pyplot as plt
from numpy.typing import NDArray
from PIL import Image
from torch import Tensor
from torchvision.transforms import ToPILImage
from torchmetrics import Metric

## Dataset
The dataset we are going to work with is the so called "MVTec", which is the de facto battlefield for anomaly detection algorithms. The dataset is composed of several product categories coming from the industry.

Reference: https://www.mvtec.com/company/research/datasets/mvtec-ad

Optionally, BTech dataset (released by _beanTech_ and University of Udine) could also be used.

Reference https://ar5iv.labs.arxiv.org/html/2104.10036

The following cell initializes the dataset and their splits. In particular, it will download the whole dataset, it will extract and convert all the images and it will split the dataset in 3 distict groups (also called splits): _training_, _validation_ and _test_. It might require some time to complete. At the end you should see a "datasets" folder in your workspace directory with all the splits well organized.

In case you already have a preprocessed version of the MVTec dataset, copy it in `./dataset` folder. In this way you will avoid all the download and processin phase. At the end, you should have a similar file structure:
```
./dataset
  |-- MVTec
  :     |-- bottle
        |     |-- ground_truth
        |     |-- train
        |     `-- test
        |-- cable
        :     :
```

In [ ]:
L.seed_everything(321)

datamodule = MVTec(
    category="cable",  # <--- We select only one category. Later you can replace this with others.
    image_size=256,
    train_batch_size=8,
    eval_batch_size=8,
    val_split_ratio=0.3,
    val_split_mode='from_test',
    test_split_ratio=0.3,
)
datamodule.prepare_data()  # <--- This line will download data, if necessary.
datamodule.setup()

# Ref: if you want to learn a bit more of the logic under the hood
# see here https://github.com/openvinotoolkit/anomalib/blob/2c2fac1a32cb2101262971a1991d625d15e61005/src/anomalib/data/base/datamodule.py#L51

### Dataset statistics

Dataset has been split into train/validation/test sets. Every sample in each
set is essentially composed of an RGB image and a binary mask (its label).

How many samples has dataset? How many of them are normal or defects?

Let's find out, by printing few statistics...

In [ ]:
# Each 'sample' is a dictionary with the following keys: 'image_path', 'label', 'image', 'mask'.
# Label is `1` for anomalies and `0` for normal samples. 'image' and 'mask' are numpy arrays.

counter = 0
for sample_id in range(len(datamodule.train_data)):
    sample = datamodule.train_data[sample_id]
    if sample["label"] == 1:
        counter += 1
print(f"Training set: {counter}/{sample_id + 1} anomalous samples")

counter = 0
for sample_id in range(len(datamodule.val_data)):
    sample = datamodule.val_data[sample_id]
    if sample["label"] == 1:
        counter += 1
print(f"Validation set: {counter}/{sample_id + 1} anomalous samples")

counter = 0
for sample_id in range(len(datamodule.test_data)):
    sample = datamodule.test_data[sample_id]
    if sample["label"] == 1:
        counter += 1
print(f"Test set: {counter}/{sample_id + 1} anomalous samples")

Note that the training set does *not* include anomalous samples. This is quite common in industry: defects
are usually rare, and hence hard to predict. In this exercise we can only count on the few anomalous samples
from the validation set to calibrate our model before testing it. 

### Look at the data!
Whenever it is possible, it is strongly recommended to look at the data with your own eyes in order to get a feeling
of the problem. The following functions will help you on this.

In [ ]:
# The following functions will be used to plot image and label side by side.
# This cell can be run as it is.

def load_image_and_resize(image_path: str, size: Tuple[int, int]) -> NDArray:
    image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    height, width = size
    return cv2.resize(image, (width, height))

def show_image_and_mask(datamodule: L.LightningDataModule, index: int | None = None, split: str = "train"):
    if split.startswith("train"):
        dataset = datamodule.train_data
    elif split.startswith("val"):
        dataset = datamodule.val_data
    elif split.startswith("test"):
        dataset = datamodule.test_data
    else:
        raise ValueError("Unknown split")
    
    if index is None:
        index = random.choice(range(len(dataset)))
    sample = dataset[index]
    gt = sample["mask"]
    im = load_image_and_resize(sample["image_path"], gt.shape[-2:])
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    fig.suptitle(sample["image_path"], fontsize=10)
    ax[0].imshow(im)
    ax[0].axis("off")
    ax[0].set_title("image")
    ax[1].imshow(gt, interpolation='nearest')
    ax[1].axis("off")
    ax[1].set_title("label")
    plt.tight_layout()
    plt.show()

Get an idea of the dataset. Plot a random sample from the training set. Run the following cell multiple times to pick other "(_image_, _label_)" pairs. As you can see the training set is composed of normal samples only (i.e. label full o 0's).

In [ ]:
%matplotlib inline
show_image_and_mask(datamodule, split="train")

### EXERCISE
Run the previous cell by changing dataset split (e.g. "val"). Run it multiple times. Since the number of anomalous samples
is quite limited you could call the function `show_image_and_mask(...)` with the extra argument `index=XXX` in order to force
a specific sample id. Try to print the IDs of the anomalous samples of the validation set.

In [ ]:
# Here few hints:
#  - Look at the function `show_image_and_mask` and see how it extracts the "sample" object.
#  - Try to iterate over the validation set keeping track of the IDs
#  - The mask of anomalous samples will have pixel greater than 0.

anomalous_ids = []
for sample_id in range(len(datamodule.val_data)):
    sample = datamodule.val_data[sample_id]
    ### Fill this part...
    # ...
    # ...

%matplotlib inline
show_image_and_mask(datamodule, index=random.choice(anomalous_ids), split="val")

## Model

Define a model (for example Padim) and fit it on the training set.

Start by defining few classic metrics. They will be measured both image-wise and pixel-wise.

In [ ]:
image_metrics = {
    "F1Score": {
        "class_path": "torchmetrics.classification.BinaryF1Score",
        "init_args": {},
    },
    "Precision": {
        "class_path": "torchmetrics.classification.BinaryPrecision",
        "init_args": {},
    },
    "Recall": {
        "class_path": "torchmetrics.classification.BinaryRecall",
        "init_args": {},
    },
}
pixel_metrics = image_metrics | {"PerRegionOverlap": {"class_path": "anomalib.metrics.PRO", "init_args": {}}}

Next, we are defining two important objects: the `model` (_Padim_ in this case) and the `engine`. The former is responsible of defining how features are extracted and classified, the latter is more a "runner" that invokes the model with the specific dataset and knows how to collect the results. Let's see them in action!

In [ ]:
# This is a simple configuration for Padim. Later in this workshop we will try to change this a bit.
model = Padim(layers=["layer1"], n_features=64)

# For the `Engine` is enough to specify our metrics while leaving all the other parameters as default.
engine = Engine(
    image_metrics=image_metrics,
    pixel_metrics=pixel_metrics,
)

The actual training process is called `fit`. In particular, a fitting loop is composed of a _training phase_ followed by a _validation phase_. Notice that we are passing the `datamodule` object and let the `engine` selecting for the required split in each sub-phase.

In [ ]:
engine.fit(model=model, datamodule=datamodule)

Now we are ready to inspect some predictions...

In [ ]:
def plot_single_prediction(predictions: List[Dict], index: int) -> None:
    batch_size = len(predictions[0]['image'])
    batch_idx = index // batch_size
    sample_idx = index % batch_size
    
    image = load_image_and_resize(
        image_path=predictions[batch_idx]["image_path"][sample_idx],
        size=predictions[batch_idx]["anomaly_maps"][sample_idx].shape[-2:],
    )
    anomaly_map = predictions[batch_idx]["anomaly_maps"][sample_idx].permute(1, 2, 0).cpu().numpy()
    heat_map = superimpose_anomaly_map(anomaly_map=anomaly_map, image=image, normalize=True)
    pred_mask = predictions[batch_idx]["pred_masks"][sample_idx].permute(1, 2, 0).cpu().numpy()
    fig, ax = plt.subplots(2, 2, figsize=(10, 10))
    fig.suptitle(predictions[batch_idx]["image_path"][sample_idx], fontsize=10)
    ax[0, 0].imshow(image)
    ax[0, 0].axis("off")
    ax[0, 0].set_title("image")
    ax[0, 1].imshow(anomaly_map)
    ax[0, 1].axis("off")
    ax[0, 1].set_title("anomaly map")
    ax[1, 0].imshow(heat_map)
    ax[1, 0].axis("off")
    ax[1, 0].set_title("heatmap")
    ax[1, 1].imshow(pred_mask, interpolation='nearest')
    ax[1, 1].axis("off")
    ax[1, 1].set_title("predicted mask")
    plt.tight_layout()
    plt.show()

In [ ]:
# Note: `engine.predict()` will use the 'test' split from the datamodule, unless specified explicitly.
# Differently from the training and validation phase, `predict` does not compute metrics, but it will
# return more elaborated outputs.
predictions = engine.predict(model=model, datamodule=datamodule, return_predictions=True)

In [ ]:
%matplotlib inline
plot_single_prediction(predictions, index=1)  # <-- Try to change the index and explore other outputs.

In [ ]:
# Note: `engine.test()` computes (and prints) the metrics.   
test_results = engine.test(model=model, datamodule=datamodule)

## Increase the depth of the backbone
Features are extracted from a pretrained backbone (in this case a Resnet18). As you may know, deep networks are composed of series of layers and each layer highlights a specific semanatic. We don't know in advance which of them work better in our problem, but we have the tools to find out.

In the following experiments we will classify the features collected at different layers and we will plot the respective metrics.

In [ ]:
results = defaultdict(list)

layer_groups = [["layer1"], ["layer2"], ["layer3"]]

# Fix the number of features (64) and increase the depth
L.seed_everything(42)
for layers in layer_groups:
    model = Padim(layers=layers, n_features=64)
    engine = Engine(
        image_metrics=image_metrics,
        pixel_metrics=pixel_metrics,
    )
    engine.fit(model=model, datamodule=datamodule)
    results_i = engine.test(model=model, datamodule=datamodule)
    for metric_name, metric_value in results_i[0].items():
        results[metric_name].append(metric_value)

In [ ]:
def get_marker_style(metric: str) -> str:
    if "image" in metric:
        color = "r"
    elif "pixel" in metric:
        color = "b"
    else:
        color = "g"
    if "F1" in metric:
        shape = "s"
    elif "Precision" in metric:
        shape = "o"
    elif "Recall" in metric:
        shape = "^"
    else:
        shape = "*"
    return f"{color}{shape}--"

In [ ]:
%matplotlib inline

fig, ax = plt.subplots(figsize=(9,5))
xticks = np.arange(1, len(list(results.values())[0]) + 1) - 0.5
for metric, result in results.items():
    ax.plot(xticks, result, get_marker_style(metric),label=metric)
ax.axis((0.3, 4, 0., 1.1))
ax.set_title('Backbone depth')
ax.set_xticks(ticks=xticks, labels=[f"{layer_group}" for layer_group in layer_groups])
ax.set_ylabel('Score')
ax.legend(loc='upper right')
plt.show()

### EXERCISE
It does not seem to be a "clear winner" between features from _Layer1_, _Layer2_ or _Layer3_.

Try to run another experiment and use a subset of features extracted from both layers. Note that the argument `layers` takes a list of strings.

## Increase the number of features

The features extracted from the backbone are (randomly) reduced in number. We could argue that the more retained features, the higher the metrics. Let's verify this argument with a simple experiment.

### EXERCISE

In [ ]:
_ = L.seed_everything(42)  # this is required for reproducibility of the random projection.

n_features_sweep = [16, 32, 64, 128]
for n_features in n_features_sweep:
    model = Padim(layers=["layer1", "layer2"], n_features=n_features)
    # ...

### EXERCISE
Try to change the seed and check if results are different.

## Tuning the threshold

You might wonder how the threshold on the anomaly maps is chosen in order to produce a binary anomaly mask. By default, the _Engine_ selects the value that maximises the F1Score. This is how the validation set is used.

Let's check how well the model behaves on the validation set.

In [ ]:
# Define a model, we'll keep this configuration throughout this section.
engine = Engine(image_metrics=image_metrics, pixel_metrics=[])
model = Padim(layers=["layer1"], n_features=64)

In [ ]:
engine.fit(model, datamodule=datamodule)

In [ ]:
# Now test the fitted model against the validation set. Instead of feeding the `datamodule`,
# pass the validation dataloader explicitly.
_ = engine.test(model=model, dataloaders=[datamodule.test_dataloader()])

If you've used the default settings for the dataset and the model you will notice an image-wise recall of **~92%**, which translates into 2 missed anomalous samples (i.e. false-negatives) out of the 27 included in the validation set.

In the real world, depending on use cases this choice might not be ideal. For instance, sometimes you prefer to sacrifice false-positives in order to reduce the number of missed anomalies.

### EXERCISE
Find the (highest) threshold for which the image-wise recall on the validation set > 95%.

In [ ]:
# Idea:
#  - let's use a PR curve in order to take the threshold that satisfies the requirement.
#  - Write a custom `Threshold` policy

# You can start from this skeleton class and fill the missing part

class MyThreshold(BinaryPrecisionRecallCurve, ManualThreshold):
    def __init__(self, default_value: float = 0.5, **kwargs) -> None:
        super().__init__(**kwargs)
        self.add_state("value", default=torch.tensor(default_value), persistent=True)
        self.value = torch.tensor(default_value)

    def compute(self) -> torch.Tensor:
        """Compute the threshold.
        
        Store the threshold into `self.value` and return it
        """
        precision: torch.Tensor
        recall: torch.Tensor
        thresholds: torch.Tensor
        precision, recall, thresholds = super().compute()  # <-- use these!

        # FILL HERE:
        # ...

    def __repr__(self) -> str:
        return f"{super().__repr__()} (value={self.value:.2f})"

In [ ]:
# Verify custom threshold, pass it explicitly to the `Engine`.
engine = Engine(
    threshold=MyThreshold(),
    image_metrics=image_metrics,
    pixel_metrics=[],
)
model = Padim(layers=["layer1"], n_features=64)
engine.fit(model=model, datamodule=datamodule)

In [ ]:
metrics_val = engine.test(model=model, dataloaders=[datamodule.val_dataloader()])

Let's see how metrics have changed on the test set.

In [ ]:
_ = engine.test(model=model, dataloaders=[datamodule.test_dataloader()])

## Cherry picking

As already stated above, it is very important to always pair numbers with visual inspection. A common practice in data-science is to manually select few examples where the model behaves particularly well or where it struggles the most. This process is called "[_cherry-picking_](https://en.wikipedia.org/wiki/Cherry_picking)".

### EXERCISE
Try to identify the missed anomalous samples from the test set and plot their anomaly maps.

## Changing the model: what about PatchCore?

Reference: https://www.amazon.science/publications/towards-total-recall-in-industrial-anomaly-detection

In [ ]:
# `Patchcore` subclasses `AnomalyModule`, like `Padim` and can be fed into an `Engine
pc = Patchcore(backbone="resnet18", layers=["layer3"], coreset_sampling_ratio=0.1)

### EXERCISE
Try with different size of the "coreset" (10% and 1%)

## Changing the model: what about FastFlow?

Reference: https://arxiv.org/pdf/2111.07677

In [ ]:
engine = Engine(
    image_metrics=image_metrics,
    pixel_metrics=pixel_metrics,
    max_epochs=1,
    check_val_every_n_epoch=None,
)
ff = Fastflow()
engine.fit(model=ff, datamodule=datamodule)

In [ ]:
_ = engine.test(model=ff, datamodule=datamodule)

Please note that the _engine_ has been configured to fit for one "epoch" only. It means that model parameters has been "adjusted" of a tiny amount to match the training set only once (actually, for a number of steps that fed the training samples only once). In this case, the pretrained model used as a starting point was sufficiently general purpose to be adequate enough for our dataset. However, we could constrain the model to adhere (i.e. _fit_) our training dataset even more (hence trading its generality in for some specialization). This can be obtained by extending the training time with more epochs.

### EXERCISE
Increment the number of number of epochs (up to 10, 20 or even more). Don't raise this value too much since it will require some time to complete. After each experiment, keep track of the metrics in a dictionary and plot them a graph. 

### EXERCISE
Take the model that has the highest recall image-wise and check which are the hardest samples to identify by plotting their anomaly mask.

## Are all those examples really necessary?

Let's see what happens when the number of training samples is progressively reduced.

### EXERCISE
Plot Precision/Recall/F1Score metrics produced by few experiemnts with (even aggressively) reduced training set. Use FastFlow (default settings) as a model and train each experiment for 5 epochs.

How do you interpret the results? Why the precision is so low compared to the recall?

In [ ]:
# Hint: the Engine can take extra arguments from the PyTorch Lightning library 
# (see here for reference: https://lightning.ai/docs/pytorch/stable/common/trainer.html#limit-train-batches)

L.seed_everything(3210)
training_set_fractions = [0.05, 0.1, 0.2, 0.5, 1.0]

# FILL HERE
# ...

## Data augmentation

As you can see from plots above, having more data can help. However, this is not always possible (either because data do not exist at all or simply because collecting more samples is too expensive). One possibility to mitigate this issue is to "artifically" augment the samples using "augmentation strategies". For example, in our scenario a 90-degree rotated sample is still a valid normal sample and hence it can be added to the training set. As you can easily imagine we could increase the dataset size of a factor of four, by simply adding rotated examples. The same goes for light color corrections (ligher/darker images), geometric transformation etc. As long as your transformed samples are normal you're safe to go.

Pay attention that in some cases augmentations could produce anomalous samples (maybe a rotated cable is considered a defect, this is dependent to the problem definitions/requirements) and they should be avoided.

Until now, default augmentations have been used. These are somehow imposed by the pretrained models used throughout this notebook. However, they can be enriched with extra operations. Augmentations are `torchvision.transforms.v2.Trasnform` objects that are stored as attributes in the dataset. In particular, default augmentations (used so far by for both train/val/test) are
```python
Compose(
      Resize(size=[256], interpolation=InterpolationMode.BILINEAR, antialias=True)
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], inplace=False)
)
```
which means that samples are first downscaled (and potentially stretched) to a resolution of 256x256 and then normalized so that pixel values are normally distributed. Obviously, normalization (like also color adjustments) is not applied to labels, but this is handled internally by the dataset. In fact, only geometric transformations are propagated to label masks.

### EXERCISE

Look at the [available augmentations](https://pytorch.org/vision/0.19/transforms.html#v2-api-reference-recommended) and extend the **training** augmentation pipeline. Before running other experiments try to inspect the effect of augmentation by plotting few examples. You might want to disable normalization (or apply a de-norm) in order to get proper color ranges.

Next, take a reduced training set (for instance 10%) and try to improve the metrics by cooking a good training pipeline.

In [ ]:
# Hints:
#  * have a look at how the `AnomalibDataModule` class definition;
#  * actual datasets are initialized during the datamodule `.setup()`
#  * transformation are applied during dataset `__getitem__()`

## Explore other approaches!
State of the art techniques can get an image F1 Score > 92% on this dataset. Do you have an idea to get close to that value? Anomalib library provides [many models](https://github.com/openvinotoolkit/anomalib/tree/main/src/anomalib/models/image) that are ready-to-use. All of them can be plugged into the _engine_, fitted and tested on the dataset.